In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string as str
import re
from textblob import TextBlob
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.metrics import classification_report

import re
tokenizer = ToktokTokenizer()
import spacy
nlp = spacy.load('en_core_web_sm', disable=['ner'])
from scipy.sparse import hstack

In [ ]:
df = pd.read_csv('train.csv')
df_ts = pd.read_csv('test.csv')

In [ ]:
df.drop(columns=['location'],inplace=True)
df_ts.drop(columns=['location'],inplace=True)

In [ ]:
df['keyword'] = df['keyword'].fillna(df['keyword'].mode()[0])
df_ts['keyword'] = df_ts['keyword'].fillna(df_ts['keyword'].mode()[0])



In [ ]:
to_be_replaced =[r"\\t|\\n|\\r","\t|\n|\r"]
values_to_replace = ["",""]

df['keyword'].replace(to_replace=to_be_replaced,value= values_to_replace,regex=True,inplace=True)
df_ts['keyword'].replace(to_replace=to_be_replaced,value= values_to_replace,regex=True,inplace=True)

df['text'].replace(to_replace=to_be_replaced,value= values_to_replace,regex=True,inplace=True)
df_ts['text'].replace(to_replace=to_be_replaced,value= values_to_replace,regex=True,inplace=True)

In [ ]:
df['keyword'].replace(r"^\s*$",np.nan,regex = True,inplace = True)
df_ts['keyword'].replace(r"^\s*$",np.nan,regex = True,inplace = True)

df['text'].replace(r"^\s*$",np.nan,regex = True,inplace = True)
df_ts['text'].replace(r"^\s*$",np.nan,regex = True,inplace = True)


In [ ]:
df['keyword'] = df['keyword'].str.encode('ascii','ignore').str.decode('ascii')
df_ts['keyword'] = df_ts['keyword'].str.encode('ascii','ignore').str.decode('ascii')


In [ ]:
df.isnull().sum()

In [ ]:
def remove_punctuation(text):
    for punctuation in str.punctuation:
        text.replace(punctuation,'')

    return text

In [ ]:
df['text'] = df['text'].apply(remove_punctuation)
df_ts['text'] = df_ts['text'].apply(remove_punctuation)

In [ ]:
df['keyword'] = df['keyword'].apply(remove_punctuation)
df_ts['keyword'] = df_ts['keyword'].apply(remove_punctuation)

In [ ]:
def remove_special_characters(text):
    text = re.sub('[^a-zA-z0-9\s]', '', text)
    return text


In [ ]:
df['text'] = df['text'].apply(remove_special_characters)
df_ts['text'] = df_ts['text'].apply(remove_special_characters)


In [ ]:
df['keyword'] = df['keyword'].apply(remove_special_characters)
df_ts['keyword'] = df_ts['keyword'].apply(remove_special_characters)

In [ ]:
def special_symboles(text):
     modified_text = re.sub(r'[\[\]\(\)\{\}]', '', text)
     return modified_text


In [ ]:
df['text'] = df['text'].apply(special_symboles)
df_ts['text'] = df_ts['text'].apply(special_symboles)


In [ ]:
df['keyword'] = df['keyword'].apply(special_symboles)
df_ts['keyword'] = df_ts['keyword'].apply(special_symboles)

In [ ]:
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r' ',text)

In [ ]:
df['text'] = df['text'].apply(remove_html)
df_ts['text'] = df_ts['text'].apply(remove_html)

In [ ]:
df['keyword'] = df['keyword'].apply(remove_html)
df_ts['keyword'] = df_ts['keyword'].apply(remove_html)

In [ ]:
def remove_url(text):
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_pattern, '', text)

In [ ]:
df['text'] = df['text'].apply(remove_url)
df_ts['text'] = df_ts['text'].apply(remove_url)

In [ ]:
df['keyword'] = df['keyword'].apply(remove_url)
df_ts['keyword'] = df_ts['keyword'].apply(remove_url)

In [ ]:
def cleans(word):
    rx = re.compile(r'\D*\d')
    if rx.match(word):
        return ''
    return word
def remove_alphanumeric(strings):
    nstrings = [" ".join(filter(None, (
    cleans(word) for word in string.split())))
    for string in strings.split()]
    str1 = ' '.join(nstrings)
    return str1

In [ ]:
df['text'] = df['text'].apply(remove_alphanumeric)
df_ts['text'] = df_ts['text'].apply(remove_alphanumeric)

In [ ]:
df['keyword'] = df['keyword'].apply(remove_alphanumeric)
df_ts['keyword'] = df_ts['keyword'].apply(remove_alphanumeric)

In [ ]:
def remove_numbers(text):
    '''digit removal'''

    text = ''.join([i for i in text if not i.isdigit()])
    return text

In [ ]:
df['text'] = df['text'].apply(remove_numbers)
df_ts['text'] = df_ts['text'].apply(remove_numbers)

In [ ]:
df['keyword'] = df['keyword'].apply(remove_numbers)
df_ts['keyword'] = df_ts['keyword'].apply(remove_numbers)

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stopwords_list = set(stopwords.words('english'))
stopwords_list.remove('no')
stopwords_list.remove('not')


In [ ]:
def custom_remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords_list]
    else:

        filtered_tokens = [token for token in tokens if token.lower() not in stopwords_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [ ]:
df['text'] = df['text'].apply(custom_remove_stopwords)
df_ts['text'] = df_ts['text'].apply(custom_remove_stopwords)

In [ ]:
df['keyword'] = df['keyword'].apply(custom_remove_stopwords)
df_ts['keyword'] = df_ts['keyword'].apply(custom_remove_stopwords)

In [ ]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [ ]:
df['text'] = df['text'].apply(lemmatize_text)
df_ts['text'] = df_ts['text'].apply(lemmatize_text)


In [ ]:
df['keyword'] = df['keyword'].apply(lemmatize_text)
df_ts['keyword'] = df_ts['keyword'].apply(lemmatize_text)

In [ ]:
list = df['keyword'].value_counts().index
list2 = df_ts['keyword'].value_counts().index
count = 0
less = 100
for i in list:
    if i in list2:
        count += 1
        print(count)
    else: less -= 1
    print('less////////',less)



In [ ]:
df = df.drop_duplicates(subset=['text'])


In [ ]:
Y_train  = df['target']
X_train = df.drop(['target','id'],axis=1)

In [ ]:
X_train.duplicated(subset=['text']).sum()


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
token_pattern = r'\b\w+\b'

vectorizer = CountVectorizer(vocabulary=None,token_pattern=token_pattern)

X_traink = vectorizer.fit_transform(X_train['keyword'])
X_trainx = vectorizer.fit_transform(X_train['text'])





In [ ]:
x_train = hstack([X_traink,X_trainx])

In [ ]:
from sklearn.model_selection import train_test_split
X_train1,X_test1,Y_train1,Y_test1 = train_test_split(x_train,Y_train)

In [ ]:
X_train1.shape ,Y_train1.shape

In [ ]:
import xgboost as xgb
model_2 = xgb.XGBRegressor ()


In [ ]:
model_2.fit(X_train1,Y_train1)

In [ ]:
y_pred = model_2.predict(X_test1)

In [ ]:
y_pred = (y_pred > 0.5).astype('int')

In [ ]:
print(classification_report(Y_test1,y_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

In [ ]:
model.fit(X_train1,Y_train1)

In [ ]:
y_pred1 = model.predict(X_test1)

In [ ]:
print(classification_report(Y_test1,y_pred1))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential


# Create a Sequential model with dropout and L2 regularization
model_4 = Sequential()
model_4.add(Dense(128, activation='relu', input_shape=(15887,)))
model_4.add(Dense(32, activation='relu'))
model_4.add(Dense(1, activation='sigmoid'))
model_4.compile(optimizer = 'adam',loss='categorical_crossentropy',metrics=['accuracy'])



In [ ]:
Y_train1 = Y_train1.to_numpy().reshape(-1, 1)


In [ ]:
X_train1.shape

In [ ]:
model_4.fit(X_train1,Y_train1,epochs=5,batch_size=10)